In [46]:
import general_robotics_toolbox as grt
from Arm_Lib import *
import numpy as np
import time

#GLOBALS
NUM_JOINTS = 5
MIN_ANGLES = np.deg2rad(np.array([-90,-90,-90,-90,-135]))
MAX_ANGLES = np.deg2rad(np.array([90,90,90,90,135]))
ZERO_CONFIG = np.deg2rad(np.array([0,0,0,0,0]))
in_2_m = 0.0254

In [ ]:
def invkin(robot, desired_pose, q_current,max_steps=200,Kp=0.3*np.eye(3),KR= 0.3*np.eye(3), tol = 1e-4):
    R_d = desired_pose.R
    p_d = desired_pose.P
    d_q = q_current
    num_j = NUM_JOINTS
    norm = grt.normalize_joints(robot,q_current,_ignore_limits=True)
    q_cur = d_q # inital guess on current joint angles
    q_cur = q_cur.reshape((num_j,1))
    iterations = 0
    converged = False
    while iterations < max_steps and not converged:
        pose = grt.fwdkin(robot,q_cur.flatten(),_ignore_limits=True)
        R_cur = pose.R
        p_cur = pose.p
        J0T = grt.robotjacobian(robot,q_cur.flatten(),_ignore_limits = True)
        # Transform jacobian to EE frame ( from base frame )
        Tr = np.zeros((5,5))
        Tr[:3,:3] = R_cur.T
        Tr[3:,3:] = R_cur.T
        J0T = np.matmul(Tr,J0T)
        ER = np.matmul(np.transpose(R_d),R_cur)
        EP = np.matmul(R_cur.T,(p_cur - p_d))

        k, theta = grt.R2rot(ER)
        s = np.sin(theta/2) * np.array(k)
        vd = np.matmul(Kp,EP)
        wd = np.matmul(KR,s)
        b = np.concatenate([wd,vd])
        delta = np.matmul(np.linalg.pinv(J0T),b)
        converged = (np.abs(np.hstack((s,EP)))<tol).all()
        if not converged:
            q_cur = q_cur - delta.reshape((num_j,1))
        iterations += 1
    

    q_normed = grt.normalize(np.arange(num_j),[np.squeeze(q_cur)])
    return converged, q_normed



In [47]:
def write_arm_deg(Arm,angles,d_time):
        """Writes angles to arm where angles
            where angles is a numpy array of 5 integers."""
        phi = angles + np.array([90,90,90,90,135])
        for i in range(1,6):
            Arm.Arm_serial_servo_write(i,phi[i-1],d_time)

In [48]:
def dofbot_robot():
    """REturns an approximate Robot instance for a DOFBOT robot"""
    x = np.array([1,0,0])
    y = np.array([0,1,0])
    z = np.array([0,0,1])
    a = np.array([0,0,0])
    
    H = np.array([z,y,y,y,z]).T
    P = np.array([4.5*z,a,3.5*z,3.5*z,3.0*z,4.5*z]).T*in_2_m
    JTs = [0,0,0,0,0]
    return grt.Robot(H,P,JTs,MIN_ANGLES,MAX_ANGLES)

In [50]:
def main():
    MEFA_BOT = dofbot_robot()
    pose = grt.fwdkin(MEFA_BOT,np.zeros(5))
    print(pose)
    
    

In [51]:
if __name__ == "__main__":
    Arm = Arm_Lib.Arm_Device()
    write_arm_deg(Arm,ZERO_CONFIG,50)
    print("--STARTING--")
    time.sleep(.2)
    main()
    print("--END--")
    del Arm
    

--STARTING--
R = [[1. 0. 0.]
     [0. 1. 0.]
     [0. 0. 1.]]
p = [0.     0.     0.4826]


--END--
